# SDP Schedule Simulation

In [ ]:
import sys
sys.path.insert(0, "../src")

from sdp_par_model.scheduling.simulation import ScheduleSimulation

## Create app object that contains methods for simulating the SDP schedule

There are no arguments when initialising the object.

In [ ]:
app = ScheduleSimulation()

## Overall observatory selection & capacities

A scenario needs to be chosen to set the telescope, total number of FLOPS, and buffer sizes. The scenario must be one of: 'low-cdr', 'mid-cdr', 'low-adjusted', 'mid-adjusted'.

In [ ]:
scenario = "mid-cdr"

app.observatory_sizes_and_rates(scenario)

## Read HPSO performance characteristics

Loads high performance science objective (HPSO) characteristics generated by the export notebook. This picks up the latest file checked into Git by default, but a csv file path can be specified.

In [ ]:
# Read csv generated from custom observations (first run Export.ipynb to generate the csv files)
##########################################
# app.read_hpso_csv(csv_file='../data/csv/custom_pipelines.csv') # should fail because no ingest pipeline
app.read_hpso_csv(csv_file='../data/csv/custom_hpsos.csv') # should pass
# app.read_hpso_csv(csv_file='../data/csv/custom_max_mid_band1.csv') # should pass
# app.read_hpso_csv(csv_file='../data/csv/max_mid_band1.csv') # should fail because the buffer is too small

# Grab the newest hpso csv file from Git
##############################
# app.read_hpso_csv(csv_file=None)

## Determine computational capacity required for realtime processing

SDP needs to be able to change the observation at arbitrary times, so enough computational resources need to be reserved to deal with the most expensive case. This is determined based on parameters already calculated.

In [ ]:
app.computational_capacity()

## Generate graph

Generates a shuffled sequence with all HPSOs appearing roughly as often as expected in a real-life schedule then a graph of tasks is created. The node info can be displayed by setting `display_node_info` to True.

Note that in contrast to Francois' scheduler, the resource usage of every task is fixed up-front, so certain key sizes are set here.

In [ ]:
# 20 days of observation
Tsequence = 20 * 24 * 3600
# Each observation must be at least 10 minutes
Tobs_min = 10 * 60
# Number of jobs to run in parallel
batch_parallelism = 2
display_node_info = False

app.generate_graph(Tsequence, Tobs_min, batch_parallelism, display_node_info)

## Sanity-check

Checking there's enough capacity to run every task in isolation and resources aren't over-used on average, in order to keep up with observations. 

This is a rough estimate of safety that also under-estimates the cost of edges in high-pressure scenarios. For example, if something needs to be kept in the buffer for longer, it has a higher footprint than estimated here.

In [ ]:
app.sanity_check()

## Schedule tasks

A task time is assigned to every node, then resource usages and edge lengths are calculated along the way.

In [ ]:
app.schedule_tasks()

## Efficiency calculations

Interactively choose capacities to see how the overall efficiency is affected.

In [ ]:
app.efficiency_calculations()

## Simulate failures

Creates a temporary capacity change then re-schedules twice (once the capacity reduces, and once it's restored) to simulate failures.

In [ ]:
app.failures()